In [1]:
from aocd import data, submit
from util import *
from dataclasses import dataclass

In [6]:
data

'364289715'

In [154]:
class Node():
    def __init__(self, value: int, cw = None, ccw = None):
        self.value = value
        if cw:
            self.cw = cw
            self.ccw = ccw
        else:
            self.cw = self
            self.ccw = self
        
    def insert_after(self, node):
        """ Inserts the chain started at node after self """
        assert self.find_value(node.value) == None, "Cant double up values, mistakes were made"
        
        old_cw = self.cw
        self.cw = node
        old_cw.ccw = node.ccw
        node.ccw.cw = old_cw
        node.ccw = self
        
    def remove_next_n(self, n: int) -> Node:
        assert n > 0
        start = self.cw
        end = start
        # identify last node in to be removed, inclusive
        for i in range(n - 1):
            end = end.cw
        
        self.cw = end.cw
        end.ccw = self
        
        start.ccw = end
        end.cw = start
        
        return start
    
    def find_next_lowest_value(self, max_value = 9) -> Node:
        mod = (max_value + 1)
        target = (self.value - 1) % mod
        while True:
            node = self.find_value(target)
            if node:
                return node
            else:
                target = (target - 1) % mod
        
    def find_value(self, value) -> Optional[Node]:
        current = self
        while True:
            if current.value == value:
                return current
            current = current.cw
            if current == self:
                # we looped and never found the value
                return None
            
        
    def to_string(self):
        output = [self.value]
        current = self.cw
        while current != self:
            output.append(current.value)
            current = current.cw
        return ", ".join(mapl(str, output))
    
    def __repr__(self):
        return f"{self.ccw.value} <- {self.value} -> {self.cw.value}"


In [155]:
def parse_data(data):
    numbers = mapl(int, list(data))
    
    start = Node(value=numbers[0])
    current = start
    # build a list of nodes
    for i in range(1, len(numbers)):
        current.insert_after(Node(value=numbers[i]))
        current = current.cw
    
    return start

In [117]:
sample_data = "389125467"

In [182]:
def apply_round(current, max_value=9):    
    next_3 = current.remove_next_n(3)
    destination = current.find_next_lowest_value(max_value)
    destination.insert_after(next_3)
    return current.cw
    

In [130]:
current = parse_data(data)
for i in range(100):
    current = apply_round(current)
print(current.to_string())

one = current.find_value(1)
result = one.to_string()[1:]


457321986


In [131]:
result

'98645732'

In [132]:
submit(result)

answer a: None
submitting for part a


That's the right answer!  You are one gold star closer to saving your vacation. [Continue to Part Two]


<Response [200]>

# Part 2

In [135]:
from tqdm import tqdm, trange

In [ ]:
ROUNDS = 10_000_000

It's impossible to run the full simulation

Just need to figure out where `[1]` ends up.

We know where 1 ends up after the first 10 are processed.

The other 1M being in order means what happens to them should be predictable.

Then can figure out what happens at the end of the list, what gets pushed over into the first 10?

#### observations

- We advance 4 positions through the sequence each time (next 3 and then +1 spot)
- Since 1 is the lowest number, it's the tipping point where we wrap to the 1M number
  - We insert infront of 1 when 1M is current
  - What gets inserted infront of 1 each time? Every 4th number?


In [193]:

# current = parse_data(data)
current = parse_data("389125467")
# current = Node(1)
for i in trange(10, 21):
    current.ccw.insert_after(Node(value=i))


100%|██████████| 11/11 [00:00<00:00, 52849.19it/s]


In [194]:
print(current.to_string())

3, 8, 9, 1, 2, 5, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20


In [195]:
for i in range(40):
    print(current.to_string())
    print(f"{i}")
    current = apply_round(current, max_value=20)


3, 8, 9, 1, 2, 5, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
0
2, 8, 9, 1, 5, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 3
1
5, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 8, 9, 1, 3, 2
2
10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 8, 9, 1, 3, 4, 6, 7, 2, 5
3
14, 15, 16, 17, 18, 19, 20, 8, 9, 11, 12, 13, 1, 3, 4, 6, 7, 2, 5, 10
4
18, 19, 20, 8, 9, 11, 12, 13, 15, 16, 17, 1, 3, 4, 6, 7, 2, 5, 10, 14
5
9, 11, 12, 13, 15, 16, 17, 19, 20, 8, 1, 3, 4, 6, 7, 2, 5, 10, 14, 18
6
15, 16, 17, 19, 20, 8, 11, 12, 13, 1, 3, 4, 6, 7, 2, 5, 10, 14, 18, 9
7
20, 8, 11, 12, 13, 1, 3, 4, 6, 7, 2, 5, 10, 14, 16, 17, 19, 18, 9, 15
8
13, 1, 3, 4, 6, 7, 2, 5, 10, 14, 16, 17, 19, 8, 11, 12, 18, 9, 15, 20
9
6, 7, 2, 5, 10, 14, 16, 17, 19, 8, 11, 12, 1, 3, 4, 18, 9, 15, 20, 13
10
10, 14, 16, 17, 19, 8, 11, 12, 1, 3, 4, 7, 2, 5, 18, 9, 15, 20, 13, 6
11
19, 8, 11, 12, 1, 3, 4, 7, 2, 5, 18, 9, 14, 16, 17, 15, 20, 13, 6, 10
12
1, 3, 4, 7, 2, 5, 18, 8, 11, 12, 9, 14, 16, 17, 15, 20, 13, 6

In [196]:
(10 + 4 * 10_000_000) % 1_000_000

10